In [15]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.cell.code_cell.rendered{width:90%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:22pt;}
div.text_cell_render.rendered_html{font-size:18pt;}
div.text_cell_render.rendered_html{font-size:15pt;}
div.output {font-size:18pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:18pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:18pt;padding:5px;}
table.dataframe{font-size:18px;}
</style>
"""))

# Attention으로 번역기 만들기
- Attention 

## 1. 패키지 import & 하이퍼파라미터
- 하이퍼파라미터 : 모델의 정확도 및 학습속도에 영향을 미치는 변수


In [5]:
import numpy as np
import pandas as pd
from time import time

from tensorflow.keras.layers import Input,LSTM,Dense,Attention, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

# 하이퍼파라미터
# 입력 ->
MY_HIDDEN = 128
MY_EPOCH = 500

## 2. 번역데이터 불러오기

In [6]:
raw = pd.read_csv('data/translate.csv', header=None)
eng_kor = raw.values.tolist() # array로 변경 -> list로 변환
print('영어 → 한국어 번역 데이터 :\n',eng_kor[:2])
print('영어 → 한국어 번역 데이터수 :',len(eng_kor))

영어 → 한국어 번역 데이터 :
 [['cold', '감기'], ['come', '오다']]
영어 → 한국어 번역 데이터수 : 110


## 3. 영어알파벳과 한글문자 리스트 만들기

In [11]:
# 영어 알파벨 리스트 만들기 (e_alpha)
e_alpha = [ c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
# print(e_alpha)
# {c:i for i,c in enumerate(e_alpha)}

# 한글 문자 리스트 만들기 (k_ch)
korean = ''.join([data[1]for data in eng_kor])
k_ch = list(set([ch for ch in korean]))
k_ch.sort()
# print(k_ch)

k_alpha = pd.read_csv('data/korean.csv', header=None)[0].tolist()

In [12]:
alpha = e_alpha + k_alpha
alpha_total_size = len(alpha)

## 4.문자당 num을 갖는 dict 만들기 

In [13]:
char_to_num = {c:i for i,c in enumerate(alpha)}
print(char_to_num)

{'S': 0, 'E': 1, 'P': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28, '가': 29, '각': 30, '간': 31, '감': 32, '개': 33, '거': 34, '것': 35, '게': 36, '계': 37, '고': 38, '관': 39, '광': 40, '구': 41, '굴': 42, '규': 43, '그': 44, '금': 45, '기': 46, '깊': 47, '나': 48, '날': 49, '남': 50, '내': 51, '넓': 52, '녀': 53, '노': 54, '놀': 55, '농': 56, '높': 57, '뉴': 58, '늦': 59, '다': 60, '단': 61, '도': 62, '동': 63, '들': 64, '람': 65, '랑': 66, '래': 67, '램': 68, '류': 69, '름': 70, '릎': 71, '리': 72, '많': 73, '망': 74, '매': 75, '머': 76, '먼': 77, '멍': 78, '메': 79, '명': 80, '모': 81, '목': 82, '무': 83, '물': 84, '미': 85, '바': 86, '반': 87, '방': 88, '번': 89, '복': 90, '부': 91, '분': 92, '붕': 93, '비': 94, '뿌': 95, '사': 96, '상': 97, '색': 98, '생': 99, '서': 100, '선': 101, '소': 102, '손': 103, '수': 104, '쉽': 105, '스': 106, '시': 107, '식': 108, '실': 109, '싸': 110,

## 5.인코더 입력, 디코더 입력, 디코더 출력
- 인코더 입력데이터 : 영어알파벳 -> 숫자 -> 원핫인코딩(110,4,171)
- 디코더 입력데이터 : 'S'+ 한글문자 -> 숫자 -> 원핫인코딩(110-데이터수,3-한글2글자+S,171-전체알파벳갯수)
- 디코더 타겟데이터 : 한글문자 + 'E' -> 숫자 shape 이 (110,3)인 list -> 110,3,1 배열로

In [14]:
def encoding(eng_kor=eng_kor):
    # 리스트로 하는 이유 : 하나씩 append하기좋음
    enc_in = []  # 인코더 입력
    dec_in = []  # 디코더 입력
    dec_out = [] # 디코더 출력(타겟)
    for data in eng_kor:
#       ――――――――――――――――――――――――――――――――――――――――――――――――――
        # ★★인코더 입력데이터(영어알파벳→ 숫자→ 원핫인코딩(171개))
        eng = [char_to_num[c] for c in data[0]]
        eng_one = np.eye(alpha_total_size)[eng]
#         print('영어 → 숫자 :',eng)
#         print('숫자 → 원핫인코딩(np.eye) :\n',eng_one )
        enc_in.append(eng_one)   # eng_one 의 shape : (4-영어4자리,171-총 단어수)
    
#       ――――――――――――――――――――――――――――――――――――――――――――――――――    
        # ★★디코더 입력데이터 ('S한글'→ 숫자→ 원핫인코딩)
        kor = [char_to_num[c] for c in 'S'+data[1]]
#         print('한글 → 숫자 :',kor)
#         kor_one = to_categorical(kor,num_classes=alpha_total_size)
        kor_one = np.eye(alpha_total_size)[kor]
#         print('숫자 → 원핫인코딩(np.eye) :\n',kor_one )
        dec_in.append(kor_one)   # kor_one의 shape : (3-한글2자리+s, 171-총 단어수)
    
#       ――――――――――――――――――――――――――――――――――――――――――――――――――
        # ★★ 디코더 타겟데이터('한글E'→ 숫자)
        kor = [char_to_num[c] for c in data[1]+'E']
        dec_out.append(kor)
#         print(kor)
        
    return enc_in, dec_in, dec_out

## 6.전체 입력데이터, 타겟데이터 준비 

In [24]:
x_enc, x_dec, y_dec = encoding(eng_kor)
X_enc = np.array(x_enc)
X_dec = np.array(x_dec)
# Y_dec = np.array(y_dec).reshape(-1,3,1) # 차원추가
Y_dec = np.expand_dims(y_dec, axis=-1)
# Y_dec = np.array(y_dec)[...,np.newaxis]
X_enc.shape,X_dec.shape,Y_dec.shape

((110, 4, 171), (110, 3, 171), (110, 3, 1))

## 7-1.모델 구현 (Seq2Seq)
- Dense층 - softmax 
- argmax

In [25]:
# ★★인코더 LSTM
# (110,4,171)의 4:영어 알파벳 4개 , 171-영어+한글 원핫백터인 전체 데이터 수 110개
# (4,171)짜리 행렬로 생성
ENC_IN = Input(shape=(4,alpha_total_size)) # alpha_total)size:171

# LSTM(units=MY_HIDDEN) = LSTM층 정의 , 데이터 입력받지않았음
#                      + (ENC_IN) 을 통해 데이터 입력받음  
# _ : 필요없는 더미데이터 (윗출력)
_, state_h, state_c = LSTM(units=MY_HIDDEN,
                           return_state=True,
#                            return_sequences=False # LSTM 윗 출력 안 받음
                           )(ENC_IN) #MY_HIDDEN:128 / return_stage=True h값과 c값을 받기
# ――――――――――――――――――――――――――――――――――――――――――――――――――
# ★★인코더와 디코더 연결고리 
LINK = [state_h,state_c]

# ――――――――――――――――――――――――――――――――――――――――――――――――――
# ★★디코더 LSTM
# (110, 3, 171)의 3=한글(2개)+s(1개) , 171-영어+한글 원핫백터인 전체 데이터 수 110개
# (3,171)짜리 행렬로 생성
DEC_IN = Input(shape=(3,alpha_total_size))
DEC_MID = LSTM(units=MY_HIDDEN, # 128
#                return_state=False,
               return_sequences=True,  #윗출력 받음
               )(DEC_IN,
                initial_state=LINK)

# ――――――――――――――――――――――――――――――――――――――――――――――――――
# ★★최종 출력층
DEC_OUT = Dense(units=alpha_total_size,
                activation='softmax')(DEC_MID)

# ――――――――――――――――――――――――――――――――――――――――――――――――――
# 모델 생성 /하나가 아닌 여러개 전달할 때 list[]로 작성
model = Model(inputs=[ENC_IN,DEC_IN],
              outputs=DEC_OUT)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 4, 171)]     0           []                               
                                                                                                  
 input_12 (InputLayer)          [(None, 3, 171)]     0           []                               
                                                                                                  
 lstm_10 (LSTM)                 [(None, 128),        153600      ['input_11[0][0]']               
                                 (None, 128),                                                     
                                 (None, 128)]                                                     
                                                                                            

## 7-2.모델 구현 (Attention)

<img src="C:\Users\Admin\Desktop\attention 모델 구현 구성안.PNG"
     width='400'
     style='float:left' >

In [22]:
# 교안 26p 
# 인코더입력
ENC_IN = Input(shape=(4, alpha_total_size))                           # alpha_total_size:171개

# ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
# 인코더 LSTM : 모든 LSTM 스텝 출력
ENC_OUT, state_h, state_c = LSTM(units=MY_HIDDEN, 
                                 return_sequences=True,               # LSTM 모든 스텝의 윗 출력 / 기본값 False
                                 return_state=True                    # h값과 c값을 모두 받기 
                                 )(ENC_IN)


# ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
# 디코더 입력
DEC_IN = Input(shape=(3,alpha_total_size))                            # alpha_total_size:171개

# ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
# 디코더 LSTM 
DEC_LSTM_OUT, _, _, = LSTM(units=MY_HIDDEN,                           # MY_HIDDEN : 128
                           return_sequences=True,
                           return_state=True)(DEC_IN,
                                              initial_state=[state_h,
                                                             state_c]) 
# ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
# Attention vector(매커니즘 정의)
CONTEXT_VECTOR = Attention()([DEC_LSTM_OUT,ENC_OUT])

# 컨텍스트백터와 디코더LSTM 결과를 concat
CONTEXT_AND_LSTM_OUT = Concatenate()([CONTEXT_VECTOR,DEC_LSTM_OUT])   # 두개의 입력은 리스트로 

# ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
# 출력층
OUT = Dense(units=alpha_total_size,
            activation='softmax')(CONTEXT_AND_LSTM_OUT)

# ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
# 모델 정의
model = Model(inputs=[ENC_IN,DEC_IN], outputs=OUT)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 4, 171)]     0           []                               
                                                                                                  
 input_10 (InputLayer)          [(None, 3, 171)]     0           []                               
                                                                                                  
 lstm_8 (LSTM)                  [(None, 4, 128),     153600      ['input_9[0][0]']                
                                 (None, 128),                                                     
                                 (None, 128)]                                                     
                                                                                              

## 8. 모델 학습과정 & 학습

In [26]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy']  #validation data 안 쓰기에 안 써도 됨. loss만 로그 출력 / 쓰면 loss랑 acc 같이 출력이 됨.
             )
begin = time()
#[X_enc,X_dec] = xdata, Y_dec = ydata = 3차원 이상
model.fit([X_enc,X_dec],Y_dec,
          epochs=MY_EPOCH,
          verbose=1)
end = time()

print('학습시간 :', end-begin)

Epoch 1/500
4/4 [==============================] - 3s 11ms/step - loss: 5.1153 - accuracy: 0.2182
Epoch 2/500
4/4 [==============================] - 0s 11ms/step - loss: 4.9499 - accuracy: 0.3333
Epoch 3/500
4/4 [==============================] - 0s 10ms/step - loss: 4.1447 - accuracy: 0.3333
Epoch 4/500
4/4 [==============================] - 0s 10ms/step - loss: 3.4753 - accuracy: 0.3333
Epoch 5/500
4/4 [==============================] - 0s 10ms/step - loss: 3.4218 - accuracy: 0.3333
Epoch 6/500
4/4 [==============================] - 0s 10ms/step - loss: 3.3812 - accuracy: 0.3333
Epoch 7/500
4/4 [==============================] - 0s 10ms/step - loss: 3.3476 - accuracy: 0.3333
Epoch 8/500
4/4 [==============================] - 0s 10ms/step - loss: 3.3213 - accuracy: 0.3333
Epoch 9/500
4/4 [==============================] - 0s 9ms/step - loss: 3.2966 - accuracy: 0.3333
Epoch 10/500
4/4 [==============================] - 0s 12ms/step - loss: 3.2665 - accuracy: 0.3333
Epoch 11/500
4/4 [==

4/4 [==============================] - 0s 9ms/step - loss: 1.1432 - accuracy: 0.8697
Epoch 85/500
4/4 [==============================] - 0s 9ms/step - loss: 1.1425 - accuracy: 0.8485
Epoch 86/500
4/4 [==============================] - 0s 9ms/step - loss: 1.0857 - accuracy: 0.8758
Epoch 87/500
4/4 [==============================] - 0s 8ms/step - loss: 1.0427 - accuracy: 0.8818
Epoch 88/500
4/4 [==============================] - 0s 8ms/step - loss: 1.0053 - accuracy: 0.8970
Epoch 89/500
4/4 [==============================] - 0s 8ms/step - loss: 0.9777 - accuracy: 0.9061
Epoch 90/500
4/4 [==============================] - 0s 9ms/step - loss: 0.9581 - accuracy: 0.9212
Epoch 91/500
4/4 [==============================] - 0s 9ms/step - loss: 0.9291 - accuracy: 0.9000
Epoch 92/500
4/4 [==============================] - 0s 8ms/step - loss: 0.8916 - accuracy: 0.9000
Epoch 93/500
4/4 [==============================] - 0s 9ms/step - loss: 0.8521 - accuracy: 0.9333
Epoch 94/500
4/4 [===============

4/4 [==============================] - 0s 8ms/step - loss: 0.0319 - accuracy: 1.0000
Epoch 167/500
4/4 [==============================] - 0s 9ms/step - loss: 0.0304 - accuracy: 1.0000
Epoch 168/500
4/4 [==============================] - 0s 8ms/step - loss: 0.0314 - accuracy: 1.0000
Epoch 169/500
4/4 [==============================] - 0s 8ms/step - loss: 0.0280 - accuracy: 1.0000
Epoch 170/500
4/4 [==============================] - 0s 9ms/step - loss: 0.0257 - accuracy: 1.0000
Epoch 171/500
4/4 [==============================] - 0s 8ms/step - loss: 0.0256 - accuracy: 1.0000
Epoch 172/500
4/4 [==============================] - 0s 9ms/step - loss: 0.0240 - accuracy: 1.0000
Epoch 173/500
4/4 [==============================] - 0s 8ms/step - loss: 0.0221 - accuracy: 1.0000
Epoch 174/500
4/4 [==============================] - 0s 8ms/step - loss: 0.0204 - accuracy: 1.0000
Epoch 175/500
4/4 [==============================] - 0s 8ms/step - loss: 0.0205 - accuracy: 1.0000
Epoch 176/500
4/4 [=====

4/4 [==============================] - 0s 9ms/step - loss: 3.0818e-04 - accuracy: 1.0000
Epoch 248/500
4/4 [==============================] - 0s 8ms/step - loss: 0.0098 - accuracy: 0.9970
Epoch 249/500
4/4 [==============================] - 0s 8ms/step - loss: 5.4642e-04 - accuracy: 1.0000
Epoch 250/500
4/4 [==============================] - 0s 8ms/step - loss: 2.4155e-04 - accuracy: 1.0000
Epoch 251/500
4/4 [==============================] - 0s 8ms/step - loss: 2.2427e-04 - accuracy: 1.0000
Epoch 252/500
4/4 [==============================] - 0s 8ms/step - loss: 2.1058e-04 - accuracy: 1.0000
Epoch 253/500
4/4 [==============================] - 0s 8ms/step - loss: 1.9714e-04 - accuracy: 1.0000
Epoch 254/500
4/4 [==============================] - 0s 8ms/step - loss: 1.8526e-04 - accuracy: 1.0000
Epoch 255/500
4/4 [==============================] - 0s 8ms/step - loss: 1.7531e-04 - accuracy: 1.0000
Epoch 256/500
4/4 [==============================] - 0s 8ms/step - loss: 1.6599e-04 - accur

4/4 [==============================] - 0s 9ms/step - loss: 6.2573e-06 - accuracy: 1.0000
Epoch 327/500
4/4 [==============================] - 0s 9ms/step - loss: 6.0438e-06 - accuracy: 1.0000
Epoch 328/500
4/4 [==============================] - 0s 9ms/step - loss: 5.8542e-06 - accuracy: 1.0000
Epoch 329/500
4/4 [==============================] - 0s 8ms/step - loss: 5.6635e-06 - accuracy: 1.0000
Epoch 330/500
4/4 [==============================] - 0s 8ms/step - loss: 5.4803e-06 - accuracy: 1.0000
Epoch 331/500
4/4 [==============================] - 0s 8ms/step - loss: 5.3087e-06 - accuracy: 1.0000
Epoch 332/500
4/4 [==============================] - 0s 9ms/step - loss: 5.1393e-06 - accuracy: 1.0000
Epoch 333/500
4/4 [==============================] - 0s 8ms/step - loss: 4.9786e-06 - accuracy: 1.0000
Epoch 334/500
4/4 [==============================] - 0s 8ms/step - loss: 4.8139e-06 - accuracy: 1.0000
Epoch 335/500
4/4 [==============================] - 0s 9ms/step - loss: 4.6878e-06 - a

4/4 [==============================] - 0s 8ms/step - loss: 1.2275e-06 - accuracy: 1.0000
Epoch 406/500
4/4 [==============================] - 0s 8ms/step - loss: 1.2120e-06 - accuracy: 1.0000
Epoch 407/500
4/4 [==============================] - 0s 8ms/step - loss: 1.1943e-06 - accuracy: 1.0000
Epoch 408/500
4/4 [==============================] - 0s 8ms/step - loss: 1.1784e-06 - accuracy: 1.0000
Epoch 409/500
4/4 [==============================] - 0s 9ms/step - loss: 1.1722e-06 - accuracy: 1.0000
Epoch 410/500
4/4 [==============================] - 0s 8ms/step - loss: 1.1592e-06 - accuracy: 1.0000
Epoch 411/500
4/4 [==============================] - 0s 8ms/step - loss: 1.1440e-06 - accuracy: 1.0000
Epoch 412/500
4/4 [==============================] - 0s 8ms/step - loss: 1.1375e-06 - accuracy: 1.0000
Epoch 413/500
4/4 [==============================] - 0s 9ms/step - loss: 1.1133e-06 - accuracy: 1.0000
Epoch 414/500
4/4 [==============================] - 0s 9ms/step - loss: 1.1032e-06 - a

4/4 [==============================] - 0s 9ms/step - loss: 6.1519e-07 - accuracy: 1.0000
Epoch 485/500
4/4 [==============================] - 0s 9ms/step - loss: 6.1050e-07 - accuracy: 1.0000
Epoch 486/500
4/4 [==============================] - 0s 9ms/step - loss: 6.0399e-07 - accuracy: 1.0000
Epoch 487/500
4/4 [==============================] - 0s 9ms/step - loss: 6.0219e-07 - accuracy: 1.0000
Epoch 488/500
4/4 [==============================] - 0s 8ms/step - loss: 5.9713e-07 - accuracy: 1.0000
Epoch 489/500
4/4 [==============================] - 0s 8ms/step - loss: 5.9279e-07 - accuracy: 1.0000
Epoch 490/500
4/4 [==============================] - 0s 9ms/step - loss: 5.8846e-07 - accuracy: 1.0000
Epoch 491/500
4/4 [==============================] - 0s 9ms/step - loss: 5.8810e-07 - accuracy: 1.0000
Epoch 492/500
4/4 [==============================] - 0s 8ms/step - loss: 5.8232e-07 - accuracy: 1.0000
Epoch 493/500
4/4 [==============================] - 0s 9ms/step - loss: 5.8015e-07 - a

In [27]:
model.evaluate([X_enc,X_dec],Y_dec)

4/4 [==============================] - 1s 5ms/step - loss: 5.5342e-07 - accuracy: 1.0000


[5.534197953238618e-07, 1.0]

## 9.모델사용

In [28]:
# 쉬운 문제
easy_test = [['cold','PP'],
            ['roof','PP'],
            ['wind','PP'],
            ['life','PP'],
            ['knee','PP']]
enc_in, dec_in, dec_out = encoding(easy_test)
enc_in = np.array(enc_in)
dec_in = np.array(dec_in)
enc_in.shape, dec_in.shape

((5, 4, 171), (5, 3, 171))

In [29]:
# 위의 문제 예측하기
pred = model.predict([enc_in,dec_in])
pred.argmax(axis=-1)

1/1 [==============================] - 1s 689ms/step


array([[ 32,  46,   1],
       [145,  93,   1],
       [ 86,  65,   1],
       [ 99,  80,   1],
       [ 83,  71,   1]], dtype=int64)

In [30]:
char_to_num['감'],alpha[32]

(32, '감')

In [31]:
# cold → 감기 ([32,46] X 이렇게 나오면 안됨)

In [32]:
for test, yhat in zip(easy_test, pred):
#     print(test[0],np.argmax(yhat,axis=-1)) #yhat.argmax(axis=-1)
    eng = test[0]
    hat = np.argmax(yhat,axis=-1)
    kor = ''.join([alpha[h] for h in hat[:-1]]) # -1을 하는 이유 문자뒤에 'E'까지 나와서
    print("{} → {}".format(eng,kor))
# pred[0].argmax(axis=-1)

cold → 감기
roof → 지붕
wind → 바람
life → 생명
knee → 무릎


In [33]:
# 어려운 문제
hard_test = [['lvoe','PP'],
            ['loev','PP'],
            ['love','PP'],
            ['olve','PP'],
            ['evol','PP']]

enc_in,dec_in, _ = encoding(hard_test)
enc_in = np.array(enc_in)
dec_in = np.array(dec_in)
pred = model.predict([enc_in,dec_in]).argmax(axis=-1)
[''.join([alpha[h] for h in hat[:-1]])for hat in pred]

1/1 [==============================] - 0s 25ms/step


['사랑', '사랑', '사랑', '사랑', '매다']

In [34]:
for test, yhat in zip(hard_test,pred):
    eng = test[0]
    kor = ''.join([alpha[h] for h in yhat[:-1]])
    print('{}→{}({})'.format(eng,kor,yhat[:-1]))

lvoe→사랑([96 66])
loev→사랑([96 66])
love→사랑([96 66])
olve→사랑([96 66])
evol→매다([75 60])
